### PoC Satellite Imagery for Road quality assessments

Install

In [ ]:
#!pip install geoplot
#!pip install geojsonio
#!pip install up42-py
#!pip install ipyleaflet

Import

In [7]:
import pandas as pd
import geopandas as gpd
import folium
import up42

### Catalog search: Area of Interest (aoi)

Read road alignment file and check crs

In [8]:
# Read road alignment 
road_alignment = gpd.read_file('./road_alignment.geojson')

In [9]:
# Check Coordinate Reference Systems
road_alignment.crs

<Derived Projected CRS: EPSG:32636>
Name: WGS 84 / UTM zone 36N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 30°E and 36°E, northern hemisphere between equator and 84°N, onshore and offshore. Belarus. Cyprus. Egypt. Ethiopia. Finland. Israel. Jordan. Kenya. Lebanon. Moldova. Norway. Russian Federation. Saudi Arabia. Sudan. Syria. Turkey. Uganda. Ukraine.
- bounds: (30.0, 0.0, 36.0, 84.0)
Coordinate Operation:
- name: UTM zone 36N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
# Transform coordinate system
road_alignment = road_alignment.to_crs('EPSG:4326')

Buffer around road alignment

In [16]:
# Length of one degree of longitude at the equator (in kilometers)
length_of_degree_at_equator = 111.0  # Approximately 111 kilometers

# Desired buffer width (in meters)
corridor_wide = 500.0  # 500 meters

# Convert desired buffer width from meters to kilometers
corridor_wide_km = corridor_wide / 1000

# Calculate buffer distance width in degrees
aoi_buffer_degrees = (corridor_wide_km / length_of_degree_at_equator)/2

# Output the buffer distance in degrees
print("Buffer distance in degrees:", aoi_buffer_degrees)


Buffer distance in degrees: 0.0022522522522522522


In [17]:
# Business Specification: corridor that is 500m wide (250m buffer)

# Create a buffer around the road
buffer_distance = aoi_buffer_degrees  
road_alignment_buffer = road_alignment.buffer(buffer_distance)

# Convert GeoDataFrame to GeoJSON object
geojson_data_buffer = road_alignment_buffer.to_json()
geojson_data_line = road_alignment.to_json()

C:\Users\Nisleida Morales\AppData\Local\Temp\ipykernel_25148\3074326009.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  road_alignment_buffer = road_alignment.buffer(buffer_distance)


Visualize aoi

In [ ]:
# Create a base map
m = folium.Map(location=[road_alignment.geometry.centroid.y.mean(), road_alignment.geometry.centroid.x.mean()], zoom_start=10)

# Add the buffer layer to the map
folium.GeoJson(geojson_data_buffer, name='Buffer').add_to(m)

# Add the road line layer to the map
folium.GeoJson(geojson_data_line, name='Road Alignment').add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Show the map
m

Save aoi as geojson

In [ ]:
# Convert the buffer to a GeoDataFrame
road_alignment_buffer_gdf = gpd.GeoDataFrame(geometry=road_alignment_buffer, crs='EPSG:4326')

# Save the buffer as a GeoJSON file
aoi = 'road_alignment_buffer.geojson'
road_alignment_buffer_gdf.to_file(aoi, driver='GeoJSON')

## Catalog search: available catalog data with UP42 Python SDK

Authentication in up42

In [ ]:
# Authenticate with up42 credentials file. Register: https://console.up42.com/register

# SDK documentation https://sdk.up42.com/

up42.authenticate(cfg_file="credentials.json")

In [ ]:
# Initialize Catalog
catalog = up42.initialize_catalog()

# Available data collections
collection_list = catalog.get_collections()

# Convert list of dictionaries to DataFrame
collection_df = pd.DataFrame(collection_list)

# Display DataFrame
collection_df
collection_df.columns
collection_df['productType'].unique()



In [ ]:
collection_df.columns

In [ ]:
#collection_df

In [ ]:
def min_resolution(row):
    min_value = row['resolutionValue']['minimum']  # Obtiene el valor mínimo de la estructura de datos
    if min_value is not None:
        return min_value * 100
    else:
        return None  # Retorna None si no hay un valor mínimo definido

# Aplica la función a la columna 'resolutionValue' para crear la nueva columna 'min_resolution_cm'
collection_df['min_resolution_cm'] = collection_df.apply(min_resolution, axis=1).astype(float)


In [ ]:
len(collection_df)


In [ ]:
# Count distinct values in the column 'productType'
distinct_product_types = collection_df['productType'].value_counts()

# Display the count of distinct values
print(distinct_product_types)


In [ ]:
collection_df

In [ ]:
collection_df['min_resolution_cm'] = pd.to_numeric(collection_df['min_resolution_cm'], errors='coerce')

filtered_df = collection_df[(collection_df['min_resolution_cm'] <= 50.0)]
filtered_df = filtered_df[(filtered_df['productType'] != 'ELEVATION')]
filtered_df = filtered_df[(filtered_df['restricted'] == False)]
filtered_df
#len(filtered_df)
column_array = filtered_df['name'].values
column_array

In [ ]:
filtered_df['hostName'].unique()

In [ ]:
len(filtered_df)

In [ ]:
filtered_df['isIntegrated'].unique()

In [ ]:
aoi = up42.read_vector_file(aoi, as_dataframe=False)

In [ ]:

search_parameters = catalog.construct_search_parameters(
    collections=['pneo'], #name
    geometry = aoi,
    start_date="2023-05-01",
    end_date="2023-09-30",
    max_cloudcover=20,
    limit=10,
)
search_results_json = catalog.search((search_parameters), as_dataframe=False)
search_results_df = catalog.search((search_parameters), as_dataframe=True)
search_results_df
image_id = search_results_df['id'].values
image_id

In [ ]:
search_results_df

In [ ]:
catalog.download_quicklooks(
    image_ids=image_id,
    collection='pneo',
    output_directory="./quicklooks/",
)

Export search results to a vector file that can be ingested into a GIS or plotted in jupyter
notebook

3. Estimate the cost

In [ ]:
catalog.map_quicklooks(scenes=search_results_df, aoi=aoi)

In [ ]:
# Place and track the order of your selected scene
order_parameters = catalog.construct_order_parameters(
    data_product_id=data_product_id, image_id=search_results.id[0])
catalog.estimate_order(order_parameters)
order = catalog.place_order(order_parameters, track_status=True)

In [ ]:
https://github.com/up42/documentation-notebooks/blob/main/processing/requests_pansharpening.ipynb

In [ ]:
# Stream cloud-native files directly for your use case
asset = up42.initialize_order(order_id=order.order_id).get_assets()[0]
stac_items = asset.stac_items
asset.get_stac_asset_url(stac_asset=stac_items[0].assets.get("b02.tiff"))

coverage in square kilometers and in percentage

suggestions for the areas that are not covered by API integrated catalog data.